In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
pd.options.mode.chained_assignment = None  # default='warn'
from read_HiC import name_chromosomes

## BIN NUCLEAR RNAseq DATA FOR GENE ABUNDANCE 

In this notebook we read processed nuclear RNAseq data from Stevens et al. (3D structures of individual mammalian genomes studied by single-cell Hi-C) and we bin it in order to couple it with topological information with the same resolution.

Load gene data

In [3]:
#Load data
data_abundance = pd.read_csv('rna data/GSM2123562_haploid_mES_nuclear_abundance_genes_ENSMUSG.tsv',
                             sep = '\t')
chrom_info = pd.read_csv('rna data/GSE80280_corr_Expression_Chd4.txt', sep = '\t')

#Select relevant columns from chrom_info
info = {'index': chrom_info['gene_id'], 'start position':chrom_info['start_position'],
        'end position':chrom_info['end_position'], 'chromosome': chrom_info['chromosome_name'], 
        'strand': chrom_info['strand']}

info_df= pd.DataFrame(info)

#Merge the two datasets
data_abundance = data_abundance.reset_index()
RNA_seq = pd.merge(data_abundance, info_df, on='index')

Set bin sizes (in bp units). Recommended: 1Mb (1000000)

In [4]:
chrom_number=20
resolution = 1000000

chr_vec=name_chromosomes(chrom_number)
chr_vec=['{}'.format(chrom[3:]) for chrom in chr_vec]
length_chromosomes = pd.read_csv('rna data/length_chromosomes_Mb.txt', header=None)
length_chromosomes = np.array(length_chromosomes.iloc[0])+ np.ones(chrom_number)
#Convert length from Mb to bp
length_chromosomes = length_chromosomes* np.ones(len(length_chromosomes))* 1000000

Bin abundance data

In [6]:
for t, chrom in enumerate(chr_vec):
    Abundance_chrom = RNA_seq[RNA_seq['chromosome']==chrom]
    
    #Find genes middle point
    length = (Abundance_chrom['end position'] -  Abundance_chrom['start position'])//2
    Abundance_chrom['position']=  Abundance_chrom['start position'] + length
    Abundance_chrom = Abundance_chrom.sort_values(by=['position'])
    
    #Create bins
    end = length_chromosomes[t]
    bins=np.arange(0, end, resolution)
    n_bins=len(bins)
    
    #Bin abundance
    Abundance_binned=np.zeros(n_bins)
    for j in range(n_bins-1):
        select_chr= Abundance_chrom[(Abundance_chrom['position']< bins[j+1]) & (Abundance_chrom['position']> bins[j])]
        Abundance_binned[j]=np.sum(select_chr['abundance'])
        
    #Save binned abundance
    df_bin_abundance={'Abundance': Abundance_binned, 'Bins': bins}
    df_bin_abundance=pd.DataFrame(df_bin_abundance)
    df_bin_abundance.to_csv('rna data/binned abundance/resolution 1Mb/abundance_chr{}.csv'.format(chrom))
    print(chrom)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
X
